<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/SN_web_lightmode.png" width="300">
</center>


<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes



## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [2]:
# This lab requires 'httr' and 'rvest'packages, which are already pre-loaded into this lab environment.
# However, if you are working on your local RStudio, please uncomment the below codes and install the packages.

#install.packages("httr")
#install.packages("rvest")

In [17]:
library(rvest)

if (!require(httr)) {
  install.packages("httr")
  library(httr)
}

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this 

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [21]:

get_wiki_covid19_page <- function() {
    
  # Our target COVID-19 wiki page URL is: https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country  
  # Which has two parts: 
    # 1) base URL `https://en.wikipedia.org/w/index.php  
    # 2) URL parameter: `title=Template:COVID-19_testing_by_country`, seperated by question mark ?
    
  # Wiki page base
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
    
  # You will need to create a List which has an element called `title` to specify which page you want to get from Wiki
  # in our case, it will be `Template:COVID-19_testing_by_country`
  query_params <- list(title = "Template:COVID-19_testing_by_country")
    
  # - Use the `GET` function in httr library with a `url` argument and a `query` arugment to get a HTTP response
  response <- GET(url = wiki_base_url, query = query_params)
    
  # Use the `return` function to return the response
  return(response)
}




Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [25]:
# Call the get_wiki_covid19_page function and print the response
response <- get_wiki_covid19_page()
print(paste("Status code:", status_code(response)))
html_content <- content(response, "text")
print(substr(html_content, 1, 500))

[1] "Status code: 200"
[1] "<!DOCTYPE html>\n<html class=\"client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clien"


## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [38]:
# Get the root html node from the http response in task 1 
html <- read_html(html_content)

Get the tables in the HTML root node using `html_nodes` function.


In [39]:
# Get the table node from the root html node
table_nodes <- html_nodes(html, "table")

Read the specific table from the multiple tables in the `table_node` using the `html_table` function and convert it into dataframe using `as.data.frame`

_Hint:- Please read the `table_node` with index 2(ex:- table_node[2])._


In [41]:
# Read the table node and convert it into a data frame
covid_table <- html_table(table_nodes[2], fill = TRUE)
covid_df <- as.data.frame(covid_table)

print(head(covid_df, n = 3))

  Country.or.region     Date.a.  Tested Units.b. Confirmed.cases.
1       Afghanistan 17 Dec 2020 154,767  samples           49,621
2           Albania 18 Feb 2021 428,654  samples           96,838
3           Algeria  2 Nov 2020 230,553  samples           58,574
  Confirmed..tested.. Tested..population.. Confirmed..population..   Ref.
1                32.1                 0.40                    0.13    [1]
2                22.6                 15.0                     3.4    [2]
3                25.4                 0.53                    0.13 [3][4]


## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [42]:
# Print the summary of the data frame
print(summary(covid_df))

 Country.or.region    Date.a.             Tested            Units.b.        
 Length:173         Length:173         Length:173         Length:173        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 Confirmed.cases.   Confirmed..tested.. Tested..population..
 Length:173         Length:173          Length:173          
 Class :character   Class :character    Class :character    
 Mode  :character   Mode  :character    Mode  :character    
 Confirmed..population..     Ref.          
 Length:173              Length:173        
 Class :character        Class :character  
 Mode  :character        Mode  :character  


As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`. 

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [43]:
preprocess_covid_data_frame <- function(data_frame) {
  shape <- dim(data_frame)
  
  # Remove the 'World' row
  data_frame <- data_frame[!(data_frame$`Country.or.region` == "World"), ]
  
  # Remove the last row
  data_frame <- data_frame[1:(nrow(data_frame) - 1), ]
  
  # We dont need the Units and Ref columns, so can be removed
  data_frame$`Units.b.` <- NULL
  data_frame$`Ref.` <- NULL
  
  # Renaming the columns
  names(data_frame) <- c("country", "date", "tested", "confirmed", 
                         "confirmed_tested_ratio", "tested_population_ratio", "confirmed_population_ratio")
  
  # Convert column data types
  data_frame$country <- as.factor(data_frame$country)
  data_frame$date <- as.factor(data_frame$date)
  data_frame$tested <- as.numeric(gsub(",", "", data_frame$tested))
  data_frame$confirmed <- as.numeric(gsub(",", "", data_frame$confirmed))
  data_frame$confirmed_tested_ratio <- as.numeric(gsub(",", "", data_frame$confirmed_tested_ratio))
  data_frame$tested_population_ratio <- as.numeric(gsub(",", "", data_frame$tested_population_ratio))
  data_frame$confirmed_population_ratio <- as.numeric(gsub(",", "", data_frame$confirmed_population_ratio))
  
  return(data_frame)
}

Call the `preprocess_covid_data_frame` function


In [44]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
processed_covid_df <- preprocess_covid_data_frame(covid_df)

Get the summary of the processed data frame again


In [45]:
# Print the summary of the processed data frame again
print(summary(processed_covid_df))

                country             date         tested         
 Afghanistan        :  1   2 Feb 2023 :  6   Min.   :     3880  
 Albania            :  1   1 Feb 2023 :  4   1st Qu.:   512037  
 Algeria            :  1   31 Jan 2023:  4   Median :  3029859  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 31377219  
 Angola             :  1   23 Jul 2021:  3   3rd Qu.: 12386725  
 Antigua and Barbuda:  1   29 Jan 2023:  3   Max.   :929349291  
 (Other)            :166   (Other)    :149                      
   confirmed        confirmed_tested_ratio tested_population_ratio
 Min.   :       0   Min.   : 0.00          Min.   :   0.006       
 1st Qu.:   37839   1st Qu.: 5.00          1st Qu.:   9.475       
 Median :  281196   Median :10.05          Median :  46.950       
 Mean   : 2508340   Mean   :11.25          Mean   : 175.504       
 3rd Qu.: 1278105   3rd Qu.:15.25          3rd Qu.: 156.500       
 Max.   :90749469   Max.   :46.80          Max.   :3223.000       
           

After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file. 


In [49]:
# Export the data frame to a csv file
write.csv(processed_covid_df, "processed_covid_data.csv", row.names = FALSE)

Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [50]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/processed_covid_data.csv")
# File path
print(file_path)
file.exists(file_path)

[1] "/resources/RP0101EN/v2/M5_Final/processed_covid_data.csv"


[1] TRUE

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [ ]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [53]:
# Read covid_data_frame_csv from the csv file
covid_data_frame_csv <- read.csv("processed_covid_data.csv")

# Get the 5th to 10th rows, with two "country" "confirmed" columns
subset_covid_data <- covid_data_frame_csv[5:10, c("country", "confirmed")]
print(subset_covid_data)

               country confirmed
5               Angola     20981
6  Antigua and Barbuda       832
7            Argentina   9060495
8              Armenia    422963
9            Australia  10112229
10             Austria   5789991


## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [54]:
# Get the total confirmed cases worldwide
total_confirmed <- sum(covid_data_frame_csv$confirmed, na.rm = TRUE)

# Get the total tested cases worldwide
total_tested <- sum(covid_data_frame_csv$tested, na.rm = TRUE)

# Get the positive ratio (confirmed / tested)
positive_ratio <- total_confirmed / total_tested

# Print the results
print(paste("Total confirmed cases worldwide:", total_confirmed))
print(paste("Total tested cases worldwide:", total_tested))
print(paste("Overall positive ratio:", positive_ratio))


[1] "Total confirmed cases worldwide: 431434555"
[1] "Total tested cases worldwide: 5396881644"
[1] "Overall positive ratio: 0.0799414520197323"


## TASK 6: Get a country list which reported their testing data 

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [55]:
# Get the 'country' column
countries <- covid_data_frame_csv$country

# Check its class (should be Factor)
print(class(countries))

# Convert the country column into character so that you can easily sort them
countries_char <- as.character(countries)

# Sort the countries A to Z
countries_AtoZ <- sort(countries_char)

# Sort the countries Z to A
countries_ZtoA <- sort(countries_char, decreasing = TRUE)

# Print the sorted Z to A list
print(countries_ZtoA)


[1] "factor"
  [1] "Zimbabwe"               "Zambia"                 "Vietnam"               
  [4] "Venezuela"              "Uzbekistan"             "Uruguay"               
  [7] "United States"          "United Kingdom"         "United Arab Emirates"  
 [10] "Ukraine"                "Uganda"                 "Turkey"                
 [13] "Tunisia"                "Trinidad and Tobago"    "Togo"                  
 [16] "Thailand"               "Tanzania"               "Taiwan[m]"             
 [19] "Switzerland[l]"         "Sweden"                 "Sudan"                 
 [22] "Sri Lanka"              "Spain"                  "South Sudan"           
 [25] "South Korea"            "South Africa"           "Slovenia"              
 [28] "Slovakia"               "Singapore"              "Serbia"                
 [31] "Senegal"                "Saudi Arabia"           "San Marino"            
 [34] "Saint Vincent"          "Saint Lucia"            "Saint Kitts and Nevis" 
 [37] "Rwanda" 

## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [56]:
# Use a regular expression 'United.+' to find matches
united_countries <- grep("^United.+", countries_char, value = TRUE)

# Print the matched country names
print(united_countries)


[1] "United Arab Emirates" "United Kingdom"       "United States"       


## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [57]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns
country1_data <- covid_data_frame_csv[covid_data_frame_csv$country == "United States", 
                                      c("country", "confirmed", "confirmed_population_ratio")]

# Select a subset (should be only one row) of data frame based on a selected country name and columns
country2_data <- covid_data_frame_csv[covid_data_frame_csv$country == "United Kingdom", 
                                      c("country", "confirmed", "confirmed_population_ratio")]

# Print the data for the two selected countries
print(country1_data)
print(country2_data)


          country confirmed confirmed_population_ratio
166 United States  90749469                       27.4
           country confirmed confirmed_population_ratio
165 United Kingdom  22232377                       32.9


## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [58]:
# Use if-else statement
if (country1_data$confirmed_population_ratio > country2_data$confirmed_population_ratio) {
  print(paste(country1_data$country, "has a higher ratio of confirmed cases to population."))
} else {
  print(paste(country2_data$country, "has a higher ratio of confirmed cases to population."))
}


[1] "United Kingdom has a higher ratio of confirmed cases to population."


## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [ ]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
